# Fig a

In [ ]:
###############fig a######################
library(Seurat)
library(plyr)
library(patchwork)
library(dplyr)
library(pheatmap)
library(stringr)

In [ ]:
relabel <- function(R1,R2) {
  for (i in R1) {
    if (i %in% R2) {next}
    else {R1[which(R1 == i)] <- ""}
  }
  return(R1)
}
# read BCC single cell experiment
se <- readRDS("example/data/scATAC_TME_All_SummarizedExperiment.final.rds")  
all_tumor_table <- read.table("example/data/bcc_SCRIPT_enrichment.txt",comment.char = "")
all_tumor_table_norm <- all_tumor_table

In [ ]:
# transfer enrichment_table rownames to corresponding cluster and normalization
cluster <- mapvalues(rownames(all_tumor_table_norm),se@colData@listData[["Group_Barcode"]],se@colData@listData[["Clusters"]])
rownames(all_tumor_table_norm) <- paste(cluster,1:length(rownames(all_tumor_table_norm)),sep = "_") 
all_tumor_table_norm <- apply(all_tumor_table_norm, 2, function (x) (x-min(x))/(max(x)-min(x)))

In [ ]:
# build seurat object
all_tumor_seurat <- CreateSeuratObject(counts = t(all_tumor_table_norm),project = "t_cell")
all_tumor_seurat <- NormalizeData(all_tumor_seurat)
all.genes <- rownames(all_tumor_seurat)
all_tumor_seurat <- ScaleData(all_tumor_seurat, features = all.genes)

In [ ]:
# get tumor specific TF
cluster_all <- c("Cluster17","Cluster18","Cluster19","Cluster20")
cluster_normal <- paste("Cluster",c(1:16),sep = "")
bcc_tf <- vector("list",length(cluster_all))
for (i in seq_along(cluster_all)) {
  cluster_markers <- FindMarkers(all_tumor_seurat, 
                                 ident.1 = cluster_all[i], 
                                 ident.2 = cluster_normal,
                                 logfc.threshold = 1)
  bcc_tf[[i]] <- rownames(cluster_markers)
}
pre_tf <- c("BCL11A","SPI1","IRF8","ETV2","RUNX","TBX21","AP4","EOMES","NFKB","KLF4","P63","BATF","BATF3","BCL11B","RUNX1","RUNX1T1","CEBPA","CEBPB","CEBPG","KLF2","KLF3","KLF4","KLF7","KLF9")
bcc_tf <- unique(c(unlist(bcc_tf),pre_tf))

In [ ]:
# get mean of cell type enrichment score
cell_cluster <- paste("Cluster",1:20,sep = "")
cell_cluster_type <- c("Naive CD4 T","TH17","Tfh","Treg","Naive CD8 T","Th1","Memory CD8 T","CD8 TEx","Effector CD8 T","NK1","NK2","B","Plasma B","Myeloid","Endothelial","Fibroblasts","Tumor 1","Tumor 2","Tumor 3","Tumor 4")
bcc_tf <- bcc_tf[bcc_tf %in% colnames(all_tumor_table)]
tumor_type <-  as.data.frame(cbind(cluster = se@colData@listData[["Clusters"]],barcode = se@colData@listData[["Group_Barcode"]]))
tumor_cell_f <- all_tumor_table[,bcc_tf]
tumor_cell_f$barcode <- rownames(tumor_cell_f)
tem <- inner_join(tumor_cell_f,tumor_type,by = "barcode")
TF_mean <- as.data.frame(group_by(tem,cluster) %>% summarise_each(funs = mean))
rownames(TF_mean) <- mapvalues(TF_mean$cluster,cell_cluster,cell_cluster_type)
TF_mean <- TF_mean[,-c(1,length(TF_mean))]

In [ ]:
tumor_tf <- c("TBX21","JUN","FOS","EOMES","IRF8","ETV2","NKFB1","NFKB2","CEBPG","TP63")
tumor_label <- relabel(colnames(TF_mean),tumor_tf)
TF_mean <- apply(TF_mean, 2, function (x) (x-min(x))/(max(x)-min(x)))
all_tumor <- pheatmap(t(TF_mean),labels_row = tumor_label,border_color= NA,fontsize_row = 13,fontsize_col = 17,treeheight_col  = 0,treeheight_row  = 0)

# Fig b Naive T ---> TEx

In [ ]:
library(Matrix)
library(SummarizedExperiment)
library(matrixStats)
library(magrittr)
library(edgeR)
library(TTR)
set.seed(1)

## Trajectory

In [ ]:

getQuantiles <- function(x){
  trunc(rank(x))/length(x)
}

alignTrajectory <- function(df, trajectory, filter = 0.05, dof = 250, spar = 1){
  
  
  findClosest <- function(x, y, fitx, fity){
    distxy <- sqrt(rowSums(cbind((fitx - x)^2 + (fity - y)^2)))
    idxMin <- which.min(distxy)
    if(idxMin==1){
      idxMin <- idxMin + 1
    }else if(idxMin==length(fitx)){
      idxMin <- idxMin - 1
    }
    if(distxy[idxMin + 1]  < distxy[idxMin - 1]){
      diff <- 1
    }else{
      diff <- -1
    }
    data.frame(idx = idxMin, dist = distxy[idxMin], diff = diff)
  }
  
  
  dfAll <- data.frame()
  for(x in seq_along(trajectory)){
    #Subset
    dfx <- df[df$Group==trajectory[x],]
    #Mean Diff Filter
    xmean <- colMeans(dfx[,c(1,2)])
    diffx <- sqrt(colSums((t(dfx[,1:2]) - xmean)^2))
    dfx <- dfx[which(diffx <= quantile(diffx,1 - filter)),]
    #Get diff
    if(x!=length(trajectory)){
      xmean1 <- colMeans(df[df$Group==trajectory[x+1],c(1,2)])
      diffx1 <- sqrt(colSums((t(dfx[,1:2]) - xmean1)^2))
      dfx$time <- (1 - getQuantiles(diffx1)) + x
    }else{
      xmean1 <- colMeans(df[df$Group==trajectory[x-1],c(1,2)])
      diffx1 <- sqrt(colSums((t(dfx[,1:2]) - xmean1)^2))
      dfx$time <- getQuantiles(diffx1) + x
    }
    dfAll <- rbind(dfAll , dfx)
  }
  sx <- smooth.spline(dfAll$time, dfAll$x, df = dof, spar = spar)
  sy <- smooth.spline(dfAll$time, dfAll$y, df = dof, spar = spar)
  dfFit <- data.frame(x = sx[[2]], y = sy[[2]], t = seq_along(sy[[2]]))
  dfTrajectory <- df[df$Group %in% trajectory,]
  dfTime <- lapply(seq_len(nrow(dfTrajectory)), function(x){
    findClosest(dfTrajectory[x,1],dfTrajectory[x,2], dfFit[,1],dfFit[,2])
  }) %>% Reduce("rbind",.)
  dfTime$distQ <- getQuantiles(dfTime$dist)
  dfTrajectory$pseudotime <- 100*getQuantiles(dfTime$idx + matrixStats::rowProds(as.matrix(dfTime[,c("diff","distQ")])))
  
  out <- list(trajectory=dfTrajectory, fitTrajectory = dfFit)
}

trajectoryStats <- function(mat, trajectory, nSim = 1000, nFeatures = 10000){
  #--------------
  # Functions
  #--------------
  rankTrajectory <- function(mat, trajectory, n = 10000, method = "vec"){
    if(method=="vec"){
      vecRank <- c()
      for(i in seq_along(trajectory)[-length(trajectory)]){
        if(i == 1){
          rem <- c(trajectory[i])
        }else{
          rem <- c(trajectory[i], trajectory[seq(1,i-1)])
        }
        trajectoryI <- trajectory[i]
        peaksI <- head(order(mat[,trajectoryI],decreasing=TRUE), n = 10000)
        distI <- rank(sqrt(colSums((mat[peaksI,] - mat[peaksI,trajectoryI])^2))[colnames(mat) %ni% rem])
        vecRank <- c(vecRank, distI[trajectory[i+1]])
      }   
      vecRank
    }else{
      matRank <- matrix(ncol=length(trajectory)-1,nrow=ncol(mat))
      rownames(matRank) <- colnames(mat)
      colnames(matRank) <- paste0("T",seq_along(trajectory[-1]))
      for(i in seq_along(trajectory)[-length(trajectory)]){
        if(i == 1){
          rem <- c(trajectory[i])
        }else{
          rem <- c(trajectory[i], trajectory[seq(1,i-1)])
        }
        trajectoryI <- trajectory[i]
        peaksI <- head(order(mat[,trajectoryI],decreasing=TRUE), n = 10000)
        distI <- sqrt(colSums((mat[peaksI,] - mat[peaksI,trajectoryI])^2))[colnames(mat) %ni% rem]
        matRank[names(distI),i] <- distI
      }   
      matRank
    }
  }
  nullTracjectory <- function(trajectory, n = 1000){
    nullTracjectory <- list()
    while(length(nullTracjectory) < n){
      trajx <- sample(trajectory, length(trajectory))
      if(!identical(trajx, trajectory)){
        nullTracjectory[[length(nullTracjectory) + 1]] <- trajx
      }
      
    }
    nullTracjectory
  }
  #Reverse Trajectory
  trajectory <- rev(trajectory)
  # Compute Null
  message("Computing Null Trajectories...")
  nullT <- nullTracjectory(trajectory, nSim)
  pb <- txtProgressBar(min = 0, max = 100, initial = 0, style = 3)
  nullRanks <- Reduce("rbind",lapply(seq_along(nullT), function(x){
    setTxtProgressBar(pb, round(x * 100/length(nullT), 0))
    rankX <- rankTrajectory(mat, nullT[[x]], n = nFeatures)
    data.frame(mean = mean(rankX), median = median(rankX))
  }))
  # Compute Actual
  rankT <- rankTrajectory(mat, trajectory, n = nFeatures)
  rankTMat <- rankTrajectory(mat, trajectory, n = nFeatures, method = "mat")
  pvalMean <- sum(nullRanks$mean < mean(rankT)) / nrow(nullRanks)
  pvalMedian <- sum(nullRanks$median < median(rankT)) / nrow(nullRanks)
  out <- list(
    pvalMean = pvalMean,
    pvalMedian = pvalMedian,
    rankT = rankT,
    rankNull = nullRanks,
    rankTMat = rankTMat
  )
  return(out)
}

groupSums <- function (mat, groups = NULL, na.rm = TRUE, sparse = FALSE){
  stopifnot(!is.null(groups))
  stopifnot(length(groups) == ncol(mat))
  gm <- lapply(unique(groups), function(x) {
    if (sparse) {
      Matrix::rowSums(mat[, which(groups == x), drop = F], na.rm = na.rm)
    }
    else {
      rowSums(mat[, which(groups == x), drop = F], na.rm = na.rm)
    }
  }) %>% Reduce("cbind", .)
  colnames(gm) <- unique(groups)
  return(gm)
}



'%ni%' <- Negate('%in%')

In [ ]:
se <- readRDS("example/data/scATAC_TME_TCells_SummarizedExperiment.final.rds")
tcell_table <- read.table("example/project/SCRIPT/tcell/enrichment/SCRIPT_enrichment.txt",comment.char = "")
tcell_table_norm <- tcell_table
tcell_table_tra <- tcell_table

In [ ]:
# get the pseudotime series of cluster13，15，16，17
trajectory <- paste0("Cluster",c(13,15,16,17))
df <- data.frame(row.names = colnames(se), x = colData(se)$UMAP1, y = colData(se)$UMAP2, Group = colData(se)$T_Cell_Cluster)
trajAligned <- alignTrajectory(df, trajectory)
df2 <- trajAligned[[1]]
map <- mapvalues(x = rownames(tcell_table_tra), from = se@colData@listData[["Group_Barcode"]], to = se@colData@listData[["Internal_Name"]])
rownames(tcell_table_tra) <- map
df2 <- df2[order(df2$pseudotime), ] 
pseudotime_index <- rownames(df2)
test_omit_table <- tcell_table_tra[pseudotime_index,]
test_omit_table <- na.omit(test_omit_table)
psudotime_table <- df2[rownames(test_omit_table),]

In [ ]:
cluster <- mapvalues(rownames(tcell_table_norm),se@colData@listData[["Group_Barcode"]],se@colData@listData[["T_Cell_Cluster"]])
rownames(tcell_table_norm) <- paste(cluster,1:length(rownames(tcell_table_norm)),sep = "_")
tcell_table_norm <- apply(tcell_table_norm, 2, function (x) (x-min(x))/(max(x)-min(x)))
cluster_tcell_seurat <- CreateSeuratObject(counts = t(tcell_table_norm))
cluster_tcell_seurat <- NormalizeData(cluster_tcell_seurat)
all.genes <- rownames(cluster_tcell_seurat)
cluster_tcell_seurat <- ScaleData(cluster_tcell_seurat, features = all.genes)

In [ ]:
cluster_markers_naivet <- FindMarkers(cluster_tcell_seurat, 
                                 ident.1 = "Cluster13", 
                                 ident.2 = c("Cluster15","Cluster16","Cluster17"),
                                 only.pos = TRUE,
                                 logfc.threshold = 0.1)
cluster_markers_naivet <- cluster_markers_naivet[order(cluster_markers_naivet$avg_log2FC,decreasing = T), ][c(1:25),]

cluster_markers_earlyTEx <- FindMarkers(cluster_tcell_seurat, 
                                 ident.1 = "Cluster15", 
                                 ident.2 = c("Cluster13","Cluster16","Cluster17"),
                                 only.pos = TRUE,
                                 logfc.threshold = 0.1)
cluster_markers_earlyTEx <- cluster_markers_earlyTEx[order(cluster_markers_earlyTEx$avg_log2FC,decreasing = T), ][c(1:25),]

cluster_markers_interTEx <- FindMarkers(cluster_tcell_seurat, 
                                 ident.1 = "Cluster16", 
                                 ident.2 = c("Cluster13","Cluster15","Cluster17"),
                                 only.pos = TRUE,
                                 logfc.threshold = 0.1)
cluster_markers_interTEx <- cluster_markers_interTEx[order(cluster_markers_interTEx$avg_log2FC,decreasing = T), ][c(1:25),]

cluster_markers_TEx <- FindMarkers(cluster_tcell_seurat, 
                                 ident.1 = "Cluster17", 
                                 ident.2 = c("Cluster13","Cluster15","Cluster16"),
                                 only.pos = TRUE,
                                 logfc.threshold = 0.1)
cluster_markers_TEx <- cluster_markers_TEx[order(cluster_markers_TEx$avg_log2FC,decreasing = T), ][c(1:25),]
tcell_tf <- unique(c(rownames(cluster_markers_naivet),rownames(cluster_markers_earlyTEx),rownames(cluster_markers_interTEx),rownames(cluster_markers_TEx)))

In [ ]:
plot_table <- tcell_table_tra[rownames(psudotime_table),c(tcell_tf)]
plot_table <- na.omit(plot_table)
for (i in seq_along(plot_table)) {
  t <- ts(plot_table[,i])
  plot_table[,i] <- SMA(t,n=8)
  
}
plot_table <- plot_table[-c(1:7),]
plot_table <- apply(plot_table, 2, function (x) (x-min(x))/(max(x)-min(x)))

In [ ]:
tcell_cluster <- c("Cluster13","Cluster15","Cluster16","Cluster17")
tcell_cluster_type <- c("Naive CD8 T","Early TEx","Intermediate TEx","Terminal TEx")
psudotime_table$Group <- mapvalues(psudotime_table$Group,tcell_cluster,tcell_cluster_type)
plot_anno <- select(psudotime_table,Group)
plot_anno <- slice(plot_anno,8:length(rownames(plot_anno)))
ann_colors = list(
  CellType = c("Naive CD8 T" = "#00AEEC", "Early TEx" = "#b2f7ef","Intermediate TEx" = "#fcd5ce","Terminal TEx" ="#FF6D6F")
)
t <- pheatmap(t(plot_table),border=FALSE,annotation_colors = ann_colors,annotation_col =  plot_anno,legend_breaks = c(1,0),fontsize_row = 20,cellheight = 17,cellwidth = 0.055,cluster_rows = FALSE,cluster_cols = FALSE,fontsize_col = 10,show_colnames = F,treeheight_row  = 0)

# line chart

In [ ]:
line_data <- cbind(plot_table,plot_anno)
tf_mean <- group_by(line_data,Group) %>% summarise_each(funs = mean)
tf_mean <- t(tf_mean)
colnames(tf_mean) <- tf_mean[1,]
tf_mean <- as.data.frame(tf_mean[-1,c(3,1,2,4)])

tf_mean <- tf_mean[c("NR4A2","NR4A3","JUNB","BATF","FOS"),]
tf_mean_data <- gather(tf_mean,key = Period,value = `TF activity`,c(`Naive CD8 T`,`Early TEx`,`Intermediate TEx`,`Terminal TEx`))
tf_mean_data$TF <- rep(rownames(tf_mean),4)
tf_mean_data$Period <- factor(tf_mean_data$Period,levels=c("Naive CD8 T","Early TEx","Intermediate TEx","Terminal TEx"))
line <- ggplot(data=tf_mean_data,aes(x=Period, y=`TF activity`, group=TF, color=TF))+
  geom_line(size = 2)+theme_classic()+geom_point(size = 3)+theme(plot.title = element_text(hjust = 5, size = 15), 
                                                                 axis.text.y = element_text(size = 10,color='black'),
                                                                 axis.text.x = element_text(size = 16,color='black'), 
                                                                 axis.title.y = element_text(size = 20),
                                                                 axis.title.x = element_text(size = 20),
                                                                 legend.title = element_text(size = 19),
                                                                 legend.text = element_text(size = 20),)+ theme(axis.text.x = element_text(angle = 45, hjust = 1))
line

# Fig e GO term plot

In [ ]:
library(plyr)
library(ggplot2)
library(Seurat)
library(clusterProfiler)
library(org.Hs.eg.db)
library(dplyr)

In [ ]:
se <- readRDS("example/data/scATAC_TME_TCells_SummarizedExperiment.final.rds")
naive_tex_table <- read.csv("example/project/SCRIPT/tcell/target/fige/JUNB.csv")
rownames(naive_tex_table) <- naive_tex_table$X
naive_tex_table <- naive_tex_table[,-1]

In [ ]:
colnames(naive_tex_table) <- gsub("[.]","-",colnames(naive_tex_table))
rname <- mapvalues(rownames(naive_tex_table),se@colData@listData[["Group_Barcode"]],se@colData@listData[["T_Cell_Cluster"]])
rownames(naive_tex_table) <- paste(rname,1:length(rname),sep = "_")

In [ ]:
# remove the low quality target
delete <- data.frame(apply(naive_tex_table,2,sum))
colnames(delete) <- "sum"
delete$name <- rownames(delete)
delete <- delete[order(delete[,1],decreasing = FALSE),]
delete <- delete[delete$sum < 1000,]
delete_target <- rownames(delete)
naive_tex_table_norm <- naive_tex_table[,setdiff(colnames(naive_tex_table),delete_target)]

In [ ]:
naive_tex_seurat <- CreateSeuratObject(counts = t(naive_tex_table_norm))
naive_tex_seurat <- NormalizeData(naive_tex_seurat)
all.genes <- rownames(naive_tex_seurat)
naive_tex_seurat <- ScaleData(naive_tex_seurat, features = all.genes)

In [ ]:
cluster_markers_naive <- FindMarkers(naive_tex_seurat, 
                                     ident.1 = "Cluster13", 
                                     ident.2 = "Cluster17",
                                     logfc.threshold = 0.1,
                                     only.pos = TRUE)

cluster_markers_tex <- FindMarkers(naive_tex_seurat, 
                                   ident.1 = "Cluster17", 
                                   ident.2 = "Cluster13",
                                   logfc.threshold = 0.1,
                                   only.pos = TRUE)
avg_log2FC_threshold <-  0.25
p_val_threshold <- 0.01
naive_target <- rownames(cluster_markers_naive)[which(cluster_markers_naive$avg_log2FC>avg_log2FC_threshold&cluster_markers_naive$p_val<p_val_threshold)]
tex_target <- rownames(cluster_markers_tex)[which(cluster_markers_tex$avg_log2FC>avg_log2FC_threshold&cluster_markers_tex$p_val<p_val_threshold)]

In [ ]:
eg_naive <- bitr(naive_target, 
                 fromType="SYMBOL", 
                 toType=c("ENTREZID","ENSEMBL",'SYMBOL'),
                 OrgDb="org.Hs.eg.db")
go_naive <- enrichGO(eg_naive$ENTREZID, 
                     OrgDb = org.Hs.eg.db, 
                     ont='BP',
                     pAdjustMethod = 'BH',
                     pvalueCutoff = 0.1,
                     keyType = 'ENTREZID')
go_naive <- go_naive[order(go_naive[,9],decreasing = TRUE),]

In [ ]:
eg_tex <- bitr(tex_target, 
               fromType="SYMBOL", 
               toType=c("ENTREZID","ENSEMBL",'SYMBOL'),
               OrgDb="org.Hs.eg.db")
# Run GO enrichment analysis 
go_tex <- enrichGO(eg_tex$ENTREZID, 
                   OrgDb = org.Hs.eg.db, 
                   ont='BP',
                   pAdjustMethod = 'BH',
                   pvalueCutoff = 0.1,
                   keyType = 'ENTREZID')
go_tex <- go_tex[order(go_tex[,9],decreasing = TRUE),]
go_tex <- go_tex[grep("negative",go_tex$Description),][c(1:6),]

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 10,repr.plot.res = 70)
par(mar=c(3,3,0.5,0.5),mgp=c(1.5,0.5,0),bg='black')
go_naive_term=ggplot(go_naive,aes(Description,Count))+geom_point(aes(color=pvalue,size=Count))+
  labs(x = "GO Terms",y = "Gene Numbers",title = "JUNB targets in Naive T") + 
  coord_flip()+theme_bw() +
  theme(panel.grid = element_blank())+
  theme(plot.title = element_text(hjust = 0.5, size = 15), 
        axis.text.y = element_text(size = 3.9,color='black'),
        axis.text.x = element_text(size = 15,color='black'), 
        axis.title.y = element_text(size = 15),
        axis.title.x = element_text(size = 15),
        legend.title = element_text(size = 15),
        legend.text = element_text(size = 5),
        text = element_text(hjust = 0.5))+scale_color_continuous(low='#FF6D6F',high='#4ecdc4')+
  scale_x_discrete(limits = rev(go_naive$Description))+
  theme(text = element_text(size=40,family="myfont"))+
  scale_y_continuous(breaks=seq(0,25,10))+theme(legend.position=c(0.8,0.4))

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 10,repr.plot.res = 70)
par(mar=c(3,3,0.5,0.5),mgp=c(1.5,0.5,0),bg='black')
go_naive_term=ggplot(go_tex,aes(Description,Count))+geom_point(aes(color=pvalue,size=Count))+
  labs(x = "GO Terms",y = "Gene Numbers",title = "JUNB targets in TEx") + 
  coord_flip()+theme_bw() +
  theme(panel.grid = element_blank())+
  theme(plot.title = element_text(hjust = 0.5, size = 15), 
        axis.text.y = element_text(size = 3.9,color='black'),
        axis.text.x = element_text(size = 15,color='black'), 
        axis.title.y = element_text(size = 15),
        axis.title.x = element_text(size = 15),
        legend.title = element_text(size = 15),
        legend.text = element_text(size = 5),
        text = element_text(hjust = 0.5))+scale_color_continuous(low='#FF6D6F',high='#4ecdc4')+
  scale_x_discrete(limits = rev(go_tex$Description))+
  theme(text = element_text(size=40,family="myfont"))+
  scale_y_continuous(breaks=seq(0,25,10))+theme(legend.position=c(0.8,0.4))

In [ ]:
write.table (naive_target, file ="example/project/SCRIPT/tcell/naive_target.txt", sep ="", row.names = F, col.names = F, quote = F)
write.table (tex_target, file ="example/project/SCRIPT/tcell/tex_target.txt", sep ="", row.names = F, col.names = F, quote = F)

# volcano plot

In [ ]:
cluster_markers_all <- FindMarkers(naive_tex_seurat, 
                                     ident.1 = "Cluster13", 
                                     ident.2 = "Cluster17",
                                     logfc.threshold = 0.1)

In [ ]:
volcano <- cluster_markers_all[,c(1,2)]
colnames(volcano) <- c("Pvalue","Foldchange")
threshold<-as.factor((volcano$Foldchange>0.25|volcano$Foldchange<(-0.25))&volcano$Pvalue<0.01)
r03=ggplot(volcano,aes(Foldchange,-log10(Pvalue),colour=threshold))+geom_point()
r04=r03+labs(title="Volcanoplot")+theme(plot.title = element_text(hjust = 0.25))+xlim(-1,1)+theme_classic()
r05=r04+geom_vline(xintercept=c(-0.25,0.25),linetype="dotted",size=1)+geom_hline(yintercept=-log10(0.01),col="blue")

In [ ]:
r05

# Fig d

In [ ]:
library(pheatmap)

In [ ]:
plot_data <- naive_tex_table[,c(naive_target,tex_target)]
annotation_col = data.frame(
  CellType = c(rep("Naive T", 2555),rep("TEx",2912))
)
rownames(annotation_col) = rownames(plot_data)
labels_row_target =  relabel(colnames(plot_data),c("FOSL1","PRMT5","CASP8","LY9","F11R","ULBP2","AQP3","TCF7","KLF10","CCL3","SMAD3","TIGIT","CTLA4","DUSP22","CD80","IL10"))
plot_data <- apply(plot_data,2, function (x) (x-min(x))/(max(x)-min(x))*100)

In [ ]:
tg <- pheatmap(t(plot_data),labels_row = labels_row_target,color = colorRampPalette(colors = c("white","#00AEEC","#7794CF"))(100),border=FALSE,show_colnames = F,cluster_rows = FALSE,cluster_cols = FALSE,annotation_col = annotation_col)

# Fig e network plot

In [ ]:
library(Seurat)
library(clusterProfiler)
library(igraph)
library(tidyverse)
library(ggraph)
library(tidygraph)

In [ ]:
se <- readRDS("example/data/scATAC_TME_TCells_SummarizedExperiment.final.rds")
tex_use<-Read10X("example/project/SCRIPT/tcell/target/1JUNB/imputation/JUNB_10x",gene.column =1)
tex_USE<-t(tex_use)

In [ ]:
tex_ENTREZID<-c()
GO <- g_n
for (i in GO){
  t<-unique((as.character(unlist(strsplit(GO[i,8],split="/")))))
  tex_ENTREZID<-append(tex_ENTREZID,t)
  tex_ENTREZID<-unique(tex_ENTREZID)
}
tex_trans <- bitr(tex_ENTREZID, fromType="ENTREZID", toType=c("SYMBOL"), OrgDb="org.Hs.eg.db")
head(tex_trans)

In [ ]:
tex_barcode <- se@colData@listData[["Group_Barcode"]][which(se@colData@listData[["T_Cell_Cluster"]] == "Cluster17")]#只选取Cluster为TEx的细胞
use_tex<-tex_USE[rownames(tex_USE)%in%tex_barcode,]
use_qc_junb<-use_tex[rownames(use_tex)%in%rownames(as.data.frame(sort(apply(use_tex,1,sum),decreasing=TRUE)[1:500])),]#选取RPscore总和前500的细胞
TEx_use_junb<-as.data.frame(sort(apply(use_qc_junb[,1:28307],2,mean),decreasing = TRUE)[1:3000]) #选取RPscore总和前3000的target
junb_target_igra <- tex_trans$SYMBOL
TEx_use_junb$gene<-rownames(TEx_use_junb)
weight_junb<-TEx_use_junb[(TEx_use_junb$gene)%in%junb_target_igra,]
colnames(weight_junb)<-c('weight','gene')
weight_junb <- weight_junb[-c(6,7,9,10,15,17,19),]

In [ ]:
TEx_graph_junb<-data.frame(matrix(NA,12,2))
TEx_graph_junb[,1]<-c(rep('JUNB',12))
TEx_graph_junb[1:12,2]<-weight_junb$gene
TEx_graph_junb[1:12,3]<-weight_junb$weight
head(TEx_graph_junb)
colnames(TEx_graph_junb)<-c('TF',"target",'weight')
name<-data.frame(c(TEx_graph_junb$TF,TEx_graph_junb$target))
length(name$c.TEx_graph_junb.TF..TEx_graph_junb.target.)
nodes<-name%>%
  distinct()%>%
  mutate(group=c(rep('4',13)))
colnames(nodes)<-c("label","group")

nodes[(nodes$label)%in%c('JUNB'),2]='1'
nodes[(nodes$label)%in%c("TIGIT","DUSP22","CD80","CCL3","CTLA4","SMAD3"),2]='2' #文献中查阅到的为2
nodes[(nodes$label)%in%c("PARK7","ADORA2A","TRIB1","RUNX1","IL10","ADTRP"),2]='3'

edges<-TEx_graph_junb%>%
  rename(from=TF,to=target,weight=weight)


In [ ]:
net_pc<-graph_from_data_frame(
  d=edges,vertices = nodes,
  directed=TRUE)

graph_pc<-as_tbl_graph(net_pc)
options(repr.plot.width = 10, repr.plot.height = 15,repr.plot.res = 70)
par(mar=c(3,3,0.5,0.5),mgp=c(1.5,0.5,0),bg='black')

g_merge<-ggraph(graph_pc,layout="stress")+   #kk
  geom_edge_link(color='lightblue',aes(edge_width=100*weight),
                 arrow = arrow(length = unit(4, 'mm')), 
                 end_cap = circle(15, 'mm'))+
  geom_node_point(aes(color=group),size = 30,alpha=0.4)+
  geom_node_text(aes(label = name),size=8) +
  scale_color_manual(
    values = c("1"="#FDB12C","2"="#FF6D6F","3"="#b8b8ff",'4'='#cddafd')) +
  scale_edge_width(range=c(0.5,1.5))+
  theme(text = element_text(size=40,family="myfont"))+
  theme_void()+
  NoLegend()

g_merge